In [1]:
# Dependencies
import requests
import pandas as pd
from config import api_key
import time

In [2]:
# Save config information.
#https://openweathermap.org/current#zip --> INFO OF THE API HERE
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
#query_url = f"{url}appid={api_key}&units={units}&q="
query_url = f"{url}appid={api_key}&units={units}&zip="
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=1cd40ca23af8b0dd2242fb9dac528b3e&units=metric&zip=


In [3]:
zip_data = pd.read_csv("../Data/zips_output.csv")
zip_data.head()

,us_zip_code,type,primary_city,state,latitude,longitude,county,timezone,area_codes,country,estimated_population,created_at,updated_at
0,501,UNIQUE,Holtsville,NY,40.81,-73.04,Suffolk County,America/New_York,631,US,384,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
1,602,STANDARD,Aguada,PR,18.38,-67.18,NaN,0,787,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
2,601,STANDARD,Adjuntas,PR,18.16,-66.72,Adjuntas,America/Puerto_Rico,787939,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
3,544,UNIQUE,Holtsville,NY,40.81,-73.04,Suffolk County,America/New_York,631,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z
4,603,STANDARD,Aguadilla,PR,18.43,-67.15,Aguadilla,America/Puerto_Rico,787,US,0,2020-02-11T17:24:49.491Z,2020-02-11T17:24:49.491Z


In [8]:
zip_codes= zip_data["us_zip_code"]
updated_zip_codes = []
for zip_code in zip_codes:
    if len(str(zip_code)) == 3:
        updated_zip_codes.append(f'00{zip_code}')
    elif len(str(zip_code)) == 4:
        updated_zip_codes.append(f'0{zip_code}')
    else:
        updated_zip_codes.append(str(zip_code))

In [9]:

# set up lists to hold reponse info
zip_codes_final = []
lat = []
lng = []
temp = []
pressure = []
humidity = []
wind_speed = []
clouds = []
count = 0
# Loop through the list of cities and perform a request for data on each
for zip_code in updated_zip_codes:
    try:
        zip_code_varchar = str(zip_code)
        response = requests.get(query_url + zip_code_varchar).json()
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        temp.append(response['main']['temp'])
        pressure.append(response['main']['pressure'])
        humidity.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        clouds.append(response['clouds']['all'])
        zip_codes_final.append(zip_code_varchar)
        count = count + 1
        if (count % 59 == 0):
            time.sleep(2)
            print(f"sleeping{count}")
    except Exception as e:
        print(e)
        print(f"{zip_code} doesn't exist")


sleeping59
sleeping118
sleeping177
sleeping236
sleeping295
sleeping354
sleeping413
sleeping472
sleeping531
sleeping590
sleeping649
sleeping708
sleeping767
sleeping826
sleeping885
sleeping944
sleeping1003
sleeping1062
sleeping1121
sleeping1180
sleeping1239
sleeping1298
sleeping1357
sleeping1416
sleeping1475
sleeping1534
sleeping1593
sleeping1652
sleeping1711
sleeping1770
sleeping1829
sleeping1888
sleeping1947
sleeping2006
sleeping2065
sleeping2124
sleeping2183
sleeping2242
sleeping2301
sleeping2360
sleeping2419
sleeping2478
sleeping2537
sleeping2596
sleeping2655
sleeping2714
sleeping2773
sleeping2832
sleeping2891
sleeping2950
sleeping3009
sleeping3068
sleeping3127
sleeping3186
'coord'
09002 doesn't exist
'coord'
09001 doesn't exist
'coord'
09004 doesn't exist
'coord'
09003 doesn't exist
'coord'
09006 doesn't exist
'coord'
09005 doesn't exist
'coord'
09007 doesn't exist
'coord'
09010 doesn't exist
'coord'
09008 doesn't exist
'coord'
09059 doesn't exist
'coord'
09013 doesn't exist
'coord'

In [15]:
# create a data frame from cities, lat, and temp
# create a data frame from cities, lat, and temp
weather_dict = {
    "zip_code": zip_codes_final,
    "lat": lat,
    "lng": lng,
    "temp": temp,
    "pressure": pressure,
    "humidity" : humidity,
    "wind_speed" : wind_speed,
    "clouds" : clouds
}
weather_data = pd.DataFrame(weather_dict)
weather_data

,zip_code,lat,lng,temp,pressure,humidity,wind_speed,clouds
0,00501,40.8154,-73.0451,8.86,1008,51,9.26,100
1,00602,18.3800,-67.1800,28.18,1012,73,4.63,75
2,00601,18.1600,-66.7200,23.80,911,75,2.24,100
3,00544,40.8154,-73.0451,8.82,1008,51,9.26,100
4,00603,18.4300,-67.1500,27.92,1013,72,4.63,75
...,...,...,...,...,...,...,...,...
40787,99926,55.1215,-131.5790,8.81,1020,49,4.12,0
40788,99928,55.3954,-131.6754,3.72,1019,61,2.06,20
40789,99922,55.2087,-132.8259,7.97,1020,53,3.60,75
40790,99929,56.4335,-132.3529,5.69,1020,61,4.63,20


In [16]:
# Output in a csv
#weather_data["zip_code"] = weather_data["zip_code"].astype(str)
#weather_data.zip_code= weather_data.zip_code.apply('="{}"'.format)
weather_data.set_index("zip_code",inplace=True)
weather_data.to_csv("../cleaned_CSVs/weatherZipCodeData_2022_04_11.csv")